<a href="https://colab.research.google.com/github/shivaprajapati34390-netizen/ML-project/blob/main/online_payments_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# I will start this task by importing the necessary Python libraries and the dataset we need for this task:

In [ ]:
import pandas as pd
import numpy as np
data=pd.read_csv("credit_card.csv")
print(data.head())

FileNotFoundError: [Errno 2] No such file or directory: 'credit_card.csv'